In [ ]:
import os
import pickle
from sklearn.decomposition import PCA
import re
import nltk
import glob
import pandas as pd
from math import log10

from nltk.corpus import stopwords
nltk.download('stopwords', quiet=True)

stops = set(stopwords.words("english"))

count=0

In [ ]:
dictionary=dict()

In [ ]:
with open('data\\dictionary.pkl', "rb") as fp:
        dictionary = pickle.load(fp)

In [ ]:
td_matrix=dict()

In [ ]:
with open('data\\td_matrix.pkl', "rb") as fp:
        td_matrix = pickle.load(fp)

In [ ]:
td=dict()

In [ ]:
with open('data\\td.pkl', "rb") as fp:
        td = pickle.load(fp)

In [ ]:
def SplitTextIntoSentences(txt):
    dtxt=re.sub(r"\.\s|\.\n",".天官赐福,百无禁忌%@!^&ЁЪЫї",txt)
    etxt=re.sub(r"\!\s|\!\n","!天官赐福,百无禁忌%@!^&ЁЪЫї",dtxt)
    qtxt=re.sub(r"\?\s|\?\n","?天官赐福,百无禁忌%@!^&ЁЪЫї",etxt)
    sentences= re.split(r'\天\官\赐\福\,\百\无\禁\忌\%\@\!\^\&\Ё\Ъ\Ы\ї', qtxt)

    return sentences

In [ ]:
email = '[a-zA-Z0-9]+@\w+[\.\w+]+'
money='\$[0-9]+[\.[0-9]+]?[k]?'
date='[0-9]{1,2}\s[a-zA-Z]+\s[0-9]{1,4}|[0-9]{1,2}\s[0-9]{1,2}\s[0-9]{1,4}|[0-9]{1,2}\S[0-9]{1,2}\S[0-9]{1,4}|[a-zA-Z]+\s[0-9]{1,2}\S?\s[0-9]{1,4}'
phone='\([0-9]{3}\)\s[0-9]{3}\-[0-9]{4}'
words='[a-zA-z]+[\'[a-z]+]?|\S'
prop_names=''
full_exp=email+'|'+money+'|'+date+'|'+phone+'|'+words



tokenize_regex = re.compile(full_exp, re.I)

In [ ]:
def TextVectorization(txt):
  tf_matrix=pd.DataFrame(columns=dictionary.keys())
  sentences=SplitTextIntoSentences(txt)
  for s in sentences:
            tokens=tokenize_regex.findall(s)
            tf_matrix.loc[s]=0
            for token in list(dictionary):
              if token in tokens:
                  tf_matrix.at[s,token]+=1
  tf_idf_matrix=tf_matrix
  N = len(td_matrix)

  for index, row in tf_idf_matrix.iterrows():
        for token in dictionary:
              tf_idf_matrix.at[index, token] = row[token] * log10((N+1)/(td.get(token,0)+1))
              
  
  doc_vect=[]
  for token in dictionary:
        doc_vect.append(round(tf_idf_matrix[token].mean(), 3))
  
  return doc_vect

In [ ]:
docs=[]

In [ ]:
with open("data\\doc_vectorized", "rb") as file:
    docs = pickle.load(file)

In [ ]:
len(docs)

In [ ]:
topics = os.listdir('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\date\\train\\')

topics.remove('alt.atheism')
topics.remove('comp.graphics')

len(topics)

In [ ]:
def PCAtrain(rootDir):
  count = len(docs)/100
  for topic in topics:
    count += 1
    print('Topic ',topic,count)
    fcount=0
    for filename in os.listdir(rootDir + topic):
        print(filename, fcount)
        fcount += 1
        with open(rootDir + topic + '\\' + filename) as file:
            vector = TextVectorization(file.read().rstrip())
            print(vector)
        docs.append(vector)
        if fcount == 100:
            with open("data\\doc_vectorized", "wb") as file:
                pickle.dump(docs, file)
            break

In [ ]:
PCAtrain('D:\\S\\NLP\\nlp-22-autumn\\projects\\shiyan_v3\\assets\\date\\train\\')
#1 alt.atheism
#2 comp.graphics

In [ ]:
with open("data\\doc_vectorized", "rb") as file:
    vectorized_train = pickle.load(file)

In [ ]:
print(len(vectorized_train))
print(len(vectorized_train[0]))

In [ ]:

pca = PCA(n_components=100)
fitted_pca = pca.fit(vectorized_train)
with open("data\\fitted_pca", "wb") as file:
    pickle.dump(fitted_pca, file)
